In [ ]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
from HTML.preprocessing import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_train = pd.read_csv(Config.train_path)
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train = preprocessing(df_train)

In [ ]:
df_train.shape

In [ ]:
y_train_is_canceled = df_train['is_canceled'].to_numpy()
y_train_adr = df_train[df_train['adr'] < 5000]['adr'].to_numpy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
categories = df_train[sorted(list(set(nominal+ordinal) - {'company', 'adr', 'reservation_status', 'reservation_status_date', 'is_canceled', 'ID', 'index'}))].astype(str).to_numpy()
cat = enc.fit_transform(categories)

In [ ]:
cat.shape

In [ ]:
num = df_train[sorted(list(set(df_train.columns) - set(nominal+ordinal) - {'company','adr', 'reservation_status', 'reservation_status_date', 'is_canceled', 'ID', 'index'}))].to_numpy()

In [ ]:
num.shape

In [ ]:
tmp = np.concatenate((cat, num), axis=1)
x_train_is_canceled = tmp.copy()
x_train_adr = tmp[df_train['adr'] < 5000,:].copy()

In [ ]:
# dummies = pd.get_dummies(
#         data=df_train[set(df_train.columns) - {'reservation_status', 'reservation_status_date', 'is_canceled'}],
#         columns=set(nominal+ordinal) - {'reservation_status', 'reservation_status_date', 'is_canceled'},
#         drop_first=True,
#         dummy_na=True)
# dummies = dummies[sorted(set(dummies.columns) - {'index', 'ID'} - set(meaningless))]
# x_train_res = dummies[sorted(list(set(dummies.columns) - {'adr'}))].to_numpy()
# x_train_adr = dummies[dummies['adr'] < 5000][sorted(list(set(dummies.columns) - {'adr'}))].to_numpy()

In [ ]:
print('adr:', x_train_adr.shape, y_train_adr.shape)
print('res:', x_train_is_canceled.shape, y_train_is_canceled.shape)

In [ ]:
from xgboost import XGBClassifier, XGBRegressor

In [ ]:
regr = XGBRegressor(
    n_estimators=100,
    learning_rate=.3,
    max_depth=6,
    subsample=1,
    n_jobs=6,
    colsample_bytree=.7,
    random_state=1126,
    gamma=10
)
regr.fit(x_train_adr, y_train_adr)

In [ ]:
clf = XGBClassifier(
    objective='binary:logistic',
    n_estimators=1,
    learning_rate=.3,
    max_depth=6,
    subsample=.3,
    n_jobs=6,
    colsample_bytree=.9,
    random_state=1126,
    gamma=10,
)
clf.fit(x_train_is_canceled, y_train_is_canceled)

In [ ]:
pred_adr = regr.predict(x_train_adr)
prob_is_canceled = clf.predict_proba(x_train_adr)

In [ ]:
pred_is_canceled = 1 - prob_is_canceled[:,0].copy()

In [ ]:
pred_is_canceled

In [ ]:
tmp = clf.predict(x_train_adr)

In [ ]:
tmp

In [ ]:
new_df_train = df_train[df_train['adr'] < 5000].copy().reset_index()
new_df_train['revenue'] = new_df_train['adr'] * np.sum(new_df_train[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1) * (1-new_df_train['is_canceled'])
new_df_train['pred_adr'] = pred_adr
new_df_train['pred_is_canceled'] = tmp
# new_df_train['pred_is_canceled'] = pred_is_canceled
new_df_train['pred_revenue'] = new_df_train['pred_adr'] * np.sum(new_df_train[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1) * (1-new_df_train['pred_is_canceled'])
new_df_train[['revenue', 'pred_revenue']].head()

In [ ]:
plt.scatter(new_df_train['pred_revenue'], new_df_train['revenue'])
plt.show()

In [ ]:
np.mean((new_df_train['revenue'] - new_df_train['pred_revenue']).to_numpy() ** 2)

In [ ]:
np.mean(np.abs((new_df_train['revenue'] - new_df_train['pred_revenue']).to_numpy()))

In [ ]:
new_df_train = add_arrival_date(new_df_train)

In [ ]:
new_df_train_sum = new_df_train.groupby('arrival_date').sum()

In [ ]:
new_df_train_sum.head()

In [ ]:
plt.scatter(new_df_train_sum['pred_revenue'], new_df_train_sum['revenue'])
plt.show()

In [ ]:
np.mean((new_df_train_sum['revenue'] - new_df_train_sum['pred_revenue']).to_numpy() ** 2)

In [ ]:
np.mean(np.abs((new_df_train_sum['revenue'] - new_df_train_sum['pred_revenue']).to_numpy()))

In [ ]:
y_train = pd.read_csv(Config.train_label_path)
train = new_df_train_sum.merge(y_train, left_index=True, right_on='arrival_date')

In [ ]:
plt.scatter(train['pred_revenue'], train['label'], alpha=.5)
plt.scatter(train['revenue'], train['label'], alpha=.5)
plt.show()

In [ ]:
np.mean(np.abs(np.floor(train['revenue'] / 10000) - train['label']))

In [ ]:
np.mean(np.abs(np.floor(train['pred_revenue'] / 10000) - train['label']))

In [ ]:
np.mean(np.abs(train['pred_revenue'] / 10000 - train['label']))

In [ ]:
df_test = pd.read_csv(Config.test_path)
df_test.head()

In [ ]:
df_test.shape

In [ ]:
categories = df_test[sorted(list(set(nominal+ordinal) - {'company', 'adr', 'reservation_status', 'reservation_status_date', 'is_canceled', 'ID', 'index'}))].astype(str).to_numpy()
cat = enc.transform(categories)

In [ ]:
cat.shape

In [ ]:
num = df_test[sorted(list(set(df_test.columns) - set(nominal+ordinal) - {'company', 'adr', 'reservation_status', 'reservation_status_date', 'is_canceled', 'ID', 'index'}))].to_numpy()

In [ ]:
num.shape

In [ ]:
tmp = np.concatenate((cat, num), axis=1)
x_test = tmp.copy()

In [ ]:
print('test:', x_test.shape)

In [ ]:
pred_adr = regr.predict(x_test)
prob_res = clf.predict_proba(x_test)

In [ ]:
pred_res = 1- prob_res[:,0].copy()

In [ ]:
pred_res

In [ ]:
tmp = clf.predict(x_test)

In [ ]:
new_df_test = df_test.copy().reset_index()
new_df_test['pred_adr'] = pred_adr
new_df_test['pred_is_canceled'] = tmp
# new_df_test['pred_is_canceled'] = pred_res
new_df_test['pred_revenue'] = new_df_test['pred_adr'] * np.sum(new_df_test[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1) * (1-new_df_test['pred_is_canceled'])
new_df_test[['pred_adr', 'pred_is_canceled', 'pred_revenue']].head()

In [ ]:
new_df_train[['pred_adr', 'pred_is_canceled', 'pred_revenue']].describe()

In [ ]:
new_df_test[['pred_adr', 'pred_is_canceled', 'pred_revenue']].describe()

In [ ]:
new_df_test = add_arrival_date(new_df_test)

In [ ]:
new_df_test_sum = new_df_test.groupby('arrival_date').sum()

In [ ]:
new_df_test_sum.head()

In [ ]:
new_df_test_sum['pred_revenue'].describe()

In [ ]:
new_df_train_sum['revenue'].hist(density=True, alpha=.4)
new_df_train_sum['pred_revenue'].hist(density=True, alpha=.4)
new_df_test_sum['pred_revenue'].hist(density=True, alpha=.4)
plt.legend(['train (actual)', 'train (pred)', 'test'])
plt.show()

In [ ]:
test_index = new_df_test_sum.index

In [ ]:
label = np.floor(new_df_test_sum['pred_revenue'].to_numpy() / 10000)

In [ ]:
plt.hist(label, bins=np.arange(1,10))
plt.show()

In [ ]:
from HTML.save import save_prediction
save_prediction('../outputs//xgboost2_is_canceled.csv', test_index, label)